In [1]:
from pathlib import Path
import os
import sys
import glob

current_file_path = os.path.abspath('')
parent_folder_path = Path(current_file_path).parent
print(parent_folder_path)

sys.path.append(str(parent_folder_path))

c:\Workspace\Python\Machine-Learning\Documentations\Project-Experiments\PostgreSQL-Insert-Data-From-Python\Insert-Amazon-Product-Sales-2023


In [107]:
from utils.postgre_database_connection import connect_to_local_database
from tqdm import tqdm

import numpy as np
import pandas as pd
import psycopg2
import re

In [9]:
try: 
    connection = connect_to_local_database()
    cursor = connection.cursor()
    print("Berhasil Terhubung ke Database.")
    
    # cek konfigurasi apakah sudah sesuai atau belum, kalo ragu bisa di uncomment
    # cursor.close()
    # connection.close() 
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')

Berhasil Terhubung ke Database.


In [4]:
# Query untuk mendapatkan daftar tabel
GET_ALL_TABLES_NAME_QUERY = """
--sql
    select table_name
    from information_schema.tables
    where table_schema = 'amazon_products'
;
"""

cursor.execute(GET_ALL_TABLES_NAME_QUERY)
tables = cursor.fetchall()
print('Nama Tabel Berhasil Diquery Dari Database')
cursor.close()
connection.close() 

Nama Tabel Berhasil Diquery Dari Database


In [5]:
tables[:3]

[('all_grocery_and_gourmet_foods',),
 ('all_exercise_and_fitness',),
 ('all_home_and_kitchen',)]

In [6]:
tables_name = [''.join(table) for table in tables]

In [7]:
tables_name[:5]

['all_grocery_and_gourmet_foods',
 'all_exercise_and_fitness',
 'all_home_and_kitchen',
 'all_pet_supplies',
 'air_conditioners']

In [93]:
connection = connect_to_local_database()
cursor = connection.cursor()
path_save_file = os.path.join(current_file_path,'watches.csv')

COPY_TO_CSV_FORMAT = """
--sql
copy amazon_products.watches to '{}' delimiter ',' csv header;
;
"""

cursor.execute(COPY_TO_CSV_FORMAT.format(path_save_file))
# Menutup cursor dan koneksi
cursor.close()
connection.close()

In [116]:
watches_df_from_csv = pd.read_csv('watches.csv')

In [117]:
watches_df_from_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19133 entries, 0 to 19132
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            19133 non-null  object
 1   main_category   19133 non-null  object
 2   sub_category    19133 non-null  object
 3   image           19133 non-null  object
 4   link            19133 non-null  object
 5   ratings         9660 non-null   object
 6   no_of_ratings   9660 non-null   object
 7   discount_price  16469 non-null  object
 8   actual_price    18680 non-null  object
dtypes: object(9)
memory usage: 1.3+ MB


In [118]:
watches_df_from_csv.isna().sum()

name                 0
main_category        0
sub_category         0
image                0
link                 0
ratings           9473
no_of_ratings     9473
discount_price    2664
actual_price       453
dtype: int64

In [15]:
connection = connect_to_local_database()
cursor = connection.cursor()

# Query untuk mendapatkan daftar tabel
GET_ALL_TABLES_NAME_QUERY = """
--sql
    select * from amazon_products.watches
;
"""

cursor.execute(GET_ALL_TABLES_NAME_QUERY)
watches_table = cursor.fetchall()
columns_watches = [desc[0] for desc in cursor.description]
print('Values dari Tabel Watches Berhasil Diquery Dari Database')
cursor.close()
connection.close() 

Values dari Tabel Watches Berhasil Diquery Dari Database


In [80]:
watches_df = pd.DataFrame(watches_table, columns=columns_watches)

In [81]:
watches_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19133 entries, 0 to 19132
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            19133 non-null  object
 1   main_category   19133 non-null  object
 2   sub_category    19133 non-null  object
 3   image           19133 non-null  object
 4   link            19133 non-null  object
 5   ratings         19133 non-null  object
 6   no_of_ratings   19133 non-null  object
 7   discount_price  19133 non-null  object
 8   actual_price    19133 non-null  object
dtypes: object(9)
memory usage: 16.3 MB


In [105]:
watches_df[:5]

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Fastrack Reflex Play|1.3” AMOLED Display Smart...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Fastrack-Reflex-AMOLED-D...,3.6,"3,013","₹3,495","₹7,995"
1,"Fossil Gen 5E Smartwatch with AMOLED Screen, W...",accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Fossil-Smartwatch-stainl...,3.9,"12,329","₹14,795","₹18,495"
2,Fossil Analog Rose Gold Dial Women's Watch-BQ3181,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Fossil-Analog-Rose-Women...,4.5,"2,796","₹6,170","₹9,495"
3,LOUIS DEVIN LD-L144-BLU-CH Mesh Steel Chain An...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/LOUIS-DEVIN-Analogue-Col...,4.0,"5,276",₹289,"₹1,999"
4,IMPERIOUS - THE ROYAL WAY Analogue Women's Wat...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Imperious-Analog-Black-W...,3.9,"6,137",₹759,"₹1,999"


In [83]:
watches_df.nunique()

name              17917
main_category         1
sub_category          1
image             18583
link              19133
ratings              43
no_of_ratings       816
discount_price     3069
actual_price       1682
dtype: int64

In [106]:
watches_df[watches_df['ratings'].str.lower() == 'nan']

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
42,Gemini 3132QM01,accessories,Watches,https://m.media-amazon.com/images/I/81psX7pHin...,https://www.amazon.in/Gemini-3132QM01/dp/B0BB7...,NaN,NaN,"₹2,595","₹2,995"
335,Tommy Hilfiger Analog Blue Dial Women's Watch-...,accessories,Watches,https://m.media-amazon.com/images/I/81grfT9iD-...,https://www.amazon.in/Tommy-Hilfiger-Analog-Wo...,NaN,NaN,"₹7,743.98","₹11,995"
419,GUESS Analog Multicolor Dial Women's Watch-GW0...,accessories,Watches,https://m.media-amazon.com/images/I/81efd72uTW...,https://www.amazon.in/GUESS-Analog-Multicolor-...,NaN,NaN,"₹12,500","₹13,995"
485,Shocknshop Digital Sports Square Dial LED Unis...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Shocknshop-Digital-Sport...,NaN,NaN,₹299,"₹1,299"
543,Emartos Girls Kids Princess/Frozen Digital Mus...,accessories,Watches,https://m.media-amazon.com/images/I/41aDpounqT...,https://www.amazon.in/Emartos-Princess-Frozen-...,NaN,NaN,₹399,"₹1,499"
...,...,...,...,...,...,...,...,...,...
19125,NEUTRON Unique Analog White and Black Color Di...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/NEUTRON-Unique-Analog-Wh...,NaN,NaN,₹326,₹778
19126,NEUTRON Italian Designer Analog Silver and Bla...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/NEUTRON-Italian-Designer...,NaN,NaN,₹412,₹942
19127,NEUTRON Exclusive Analog White and Blue Color ...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/NEUTRON-Exclusive-Analog...,NaN,NaN,₹409,₹948
19128,NEUTRON Party Wedding Analog White and Black C...,accessories,Watches,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/NEUTRON-Party-Wedding-An...,NaN,NaN,₹393,₹914


In [113]:
watches_df.replace(r'(?i)^nan$', np.nan, regex=True, inplace=True)

In [119]:
watches_df.isna().sum() == watches_df_from_csv.isna().sum()

name              True
main_category     True
sub_category      True
image             True
link              True
ratings           True
no_of_ratings     True
discount_price    True
actual_price      True
dtype: bool

In [104]:
for column in watches_df.columns:
    data = watches_df[watches_df[column].str.match(r"^$|^\s+$|[\s+]?'+", na=False)]
    is_column_table_empty_string = data.shape[0] > 0
    print(f'kolom {column}: {is_column_table_empty_string}')

kolom name: False
kolom main_category: False
kolom sub_category: False
kolom image: False
kolom link: False
kolom ratings: False
kolom no_of_ratings: False
kolom discount_price: False
kolom actual_price: False
